In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-flood/data_source/new_data/valid/flooding/11891573354.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/860024680.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/8334177117.png
/kaggle/input/data-flood/data_source/new_data/valid/flooding/7520315140.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/1827534805.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/6106111822.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/3855038967.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/3685111553.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/2269424912.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/407960551.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/4436863796.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/147160837.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/5355684661.jpg
/kaggle/input/

In [ ]:
!pip install scikeras scikit-learn optuna
import optuna
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing import image

# Define paths
train_path = '/kaggle/input/data-flood/data_source/new_data/train'
valid_path = '/kaggle/input/data-flood/data_source/new_data/valid'
test_path = '/kaggle/input/data-flood/data_source/new_data/test'

# Create image data generators
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    rotation_range=4,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
).flow_from_directory(
    directory=train_path, target_size=(224, 224), batch_size=10
)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=10
)

def create_model(trial):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    x = Dropout(dropout_rate)(x)
    output = Dense(units=2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    for layer in base_model.layers[-12:]:
        layer.trainable = True

    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def objective(trial):
    model = create_model(trial)
    model.fit(train_batches, epochs=5, validation_data=valid_batches, verbose=2)
    _, accuracy = model.evaluate(valid_batches, verbose=0)
    return accuracy

# Re-run the study with the new data generators
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Train the best model
best_model = create_model(study.best_trial)
best_model.fit(train_batches, epochs=5, validation_data=valid_batches, verbose=2)

Found 3693 images belonging to 2 classes.
Found 1054 images belonging to 2 classes.


[I 2024-06-06 02:58:04,413] A new study created in memory with name: no-name-dba6062c-fdbb-4fc0-80ef-d4379d29fd18


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1717642699.699968     185 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717642699.726391     185 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717642746.866686     186 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717642771.301775     185 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


370/370 - 96s - 260ms/step - accuracy: 0.8313 - loss: 0.4858 - val_accuracy: 0.7125 - val_loss: 1.3556
Epoch 2/5


...

Epoch 1/5


W0000 00:00:1717643620.274619     187 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717643632.838163     188 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717643673.487512     188 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


370/370 - 67s - 182ms/step - accuracy: 0.7836 - loss: 0.5226 - val_accuracy: 0.8643 - val_loss: 0.3502
Epoch 2/5


W0000 00:00:1717643678.465465     186 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


370/370 - 55s - 148ms/step - accuracy: 0.8381 - loss: 0.4046 - val_accuracy: 0.8890 - val_loss: 0.2854
Epoch 3/5


In [ ]:
def preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

def predict_on_folder(folder_path, model, labels, output_csv_path):
    results = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith(('.jpg', '.png', '.jpeg', '.gif')):
            preprocessed_image = preprocess_image(file_path)
            predictions = model.predict(preprocessed_image)
            result = np.argmax(predictions)
            file_id = os.path.splitext(file_name)[0]
            results.append({'id': file_id, 'label': labels[result]})
    
    df_results = pd.DataFrame(results)
    df_results['label'] = df_results['label'].apply(lambda x: 1 if x == 'Flooding' else 0)
    df_results.to_csv(output_csv_path, index=False)

# Define labels
labels = ['Flooding','No_Flooding']

# Predict on test set and save results
img_folder_path = '/kaggle/input/testset-source/testset_source/new_testset'
output_csv_path = '/kaggle/working/predictions_file.csv'
predict_on_folder(img_folder_path, best_model, labels, output_csv_path)

print(f"Kết quả dự đoán đã được lưu vào {output_csv_path}")
